# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [123]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [124]:
orders = pd.read_csv(r"C:\Users\javie\Desktop\Ironhack\Tercera_Semana\Lab9\lab-pandas-calculations\your-code\Orders\Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [125]:
orders.shape

(397924, 14)

In [126]:
orders.sample(5)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
47902,78254,542862,22382,2011,2,2,11,lunch bag spaceboy design,1,2011-02-01 11:51:00,1.65,14606,United Kingdom,1.65
56342,90441,544037,21059,2011,2,2,11,party invites dinosaurs,24,2011-02-15 11:31:00,0.85,14961,United Kingdom,20.40
13323,20207,538014,72802B,2010,12,4,12,ocean scent candle in jewelled box,12,2010-12-09 12:49:00,4.25,14440,United Kingdom,51.00
67035,104477,545182,85230E,2011,2,1,14,strawbry scented votive candle,24,2011-02-28 14:48:00,0.29,14046,United Kingdom,6.96
116885,171594,551436,22374,2011,4,4,15,airline bag vintage jet set red,1,2011-04-28 15:13:00,4.25,16904,United Kingdom,4.25


In [127]:
#Check for missing data
orders.isnull().sum()

Unnamed: 0      0
InvoiceNo       0
StockCode       0
year            0
month           0
day             0
hour            0
Description     0
Quantity        0
InvoiceDate     0
UnitPrice       0
CustomerID      0
Country         0
amount_spent    0
dtype: int64

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [128]:
orders["CustomerID"].nunique()

4339

In [129]:
orders["CustomerID"].value_counts()

CustomerID
17841    7847
14911    5677
14096    5111
12748    4596
14606    2700
         ... 
15313       1
17846       1
13185       1
16953       1
16737       1
Name: count, Length: 4339, dtype: int64

In [130]:
orders[["CustomerID", "amount_spent"]]

,CustomerID,amount_spent
0,17850,15.30
1,17850,20.34
2,17850,22.00
3,17850,20.34
4,17850,20.34
...,...,...
397919,12680,10.20
397920,12680,12.60
397921,12680,16.60
397922,12680,16.60


In [131]:
#Group by Customer ID and see the sum of each customer spent
customer_spent = orders.groupby(["CustomerID"]).agg({"amount_spent":"sum", "Country":"min"}).reset_index()
#orders.groupby(["CustomerID"])["amount_spent"].agg(sum)
customer_spent.head()

,CustomerID,amount_spent,Country
0,12346,77183.60,United Kingdom
1,12347,4310.00,Iceland
2,12348,1797.24,Finland
3,12349,1757.55,Italy
4,12350,334.40,Norway


In [132]:
customer_spent.shape

(4339, 3)

In [133]:
#confirm that we dont have repeat Customer Id
customer_spent["CustomerID"].nunique()

4339

In [134]:
#Let see quartiles
customer_spent.describe()

,CustomerID,amount_spent
count,4339.000000,4339.000000
mean,15299.936852,2053.793018
std,1721.889758,8988.248381
min,12346.000000,0.000000
25%,13812.500000,307.245000
50%,15299.000000,674.450000
75%,16778.500000,1661.640000
max,18287.000000,280206.020000


In [135]:
p95 = customer_spent["amount_spent"].quantile(0.95)
p75 = customer_spent["amount_spent"].quantile(0.75)
print(p95, p75)

5840.181999999982 1661.64


In [136]:
#Segment each client by boolean mask
customer_spent["Vip_Customers"] = customer_spent["amount_spent"] >p95
customer_spent["Preferred_Customers"] = (customer_spent["amount_spent"] >= p75) & (customer_spent["amount_spent"] <= p95)
customer_spent["Normal_Customers"] = customer_spent["amount_spent"] <p75
customer_spent.head()

,CustomerID,amount_spent,Country,Vip_Customers,Preferred_Customers,Normal_Customers
0,12346,77183.60,United Kingdom,True,False,False
1,12347,4310.00,Iceland,False,True,False
2,12348,1797.24,Finland,False,True,False
3,12349,1757.55,Italy,False,True,False
4,12350,334.40,Norway,False,False,True


In [137]:
customer_spent.isnull().sum()

CustomerID             0
amount_spent           0
Country                0
Vip_Customers          0
Preferred_Customers    0
Normal_Customers       0
dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [138]:
country_most_Vip = (customer_spent.groupby(["Country"])["Vip_Customers"].agg("sum").reset_index().sort_values("Vip_Customers", ascending=False))
country_most_Vip.head()

,Country,Vip_Customers
35,United Kingdom,177
14,Germany,10
13,France,9
32,Switzerland,3
0,Australia,2


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [139]:
#Groupby Country per Custumers requiered
#Store the sum information in new column
country_most_combined = (customer_spent.groupby("Country")[["Vip_Customers","Preferred_Customers"]].agg("sum").reset_index())
country_most_combined["Preferred+Vip"] = (country_most_combined["Vip_Customers"] + country_most_combined["Preferred_Customers"])

In [140]:
#Order the values
country_most_combined.sort_values("Preferred+Vip", ascending=False).head()

,Country,Vip_Customers,Preferred_Customers,Preferred+Vip
35,United Kingdom,177,755,932
14,Germany,10,29,39
13,France,9,20,29
3,Belgium,0,11,11
32,Switzerland,3,5,8
